**This notebook is for cleaning the dataset from 2011 - 2015**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/Shareddrives/CIS520 Final Project/dataset/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# feature selection

# feature used in 2015
feat_2015_selected = ['DIABETE3','_RFHYPE5', 'TOLDHI2', '_CHOLCHK', '_BMI5', 'SMOKE100', 'CVDSTRK3', '_MICHD', '_TOTINDA', '_FRTLT1', '_VEGLT1', '_RFDRHV5', \
                 'HLTHPLN1', 'MEDCOST', 'GENHLTH', 'MENTHLTH', 'PHYSHLTH', 'DIFFWALK', 'SEX', '_AGEG5YR', 'EDUCA', 'INCOME2']

# features used in 2014
feat_2014_selected = ['GENHLTH', 'X_AGEG5YR', 'X_BMI5CAT', 'CHECKUP1', 'INCOME2', 'X_RACE', 'MSCODE', 'FLUSHOT6', 'EMPLOY1', 'SEX', 'MARITAL', 'X_EDUCAG', \
                 'SLEPTIM1', 'CVDCRHD4', 'HLTHCVR1', 'MENTHLTH', 'CHCKIDNY', 'USEEQUIP', 'X_TOTINDA', 'ADDEPEV2', 'RENTHOM1', 'EXERANY2', 'BLIND', \
                 'DECIDE', 'HLTHPLN1', 'DIABETE3', '_SMOKER3']

# intersection of features
feat_intsec_1415 = list(set(feat_2015_selected).intersection(set(feat_2014_selected)))
print('feature intersection 2014 & 2015: ', len(feat_intsec_1415))
print(feat_intsec_1415)
print()

# union of features
feat_union_1415 = list(set(feat_2015_selected).union(set(feat_2014_selected)))
print('feature union 2014 & 2015: ', len(feat_union_1415))
print(feat_union_1415)

feature intersection 2014 & 2015:  6
['GENHLTH', 'SEX', 'DIABETE3', 'MENTHLTH', 'HLTHPLN1', 'INCOME2']

feature union 2014 & 2015:  43
['SLEPTIM1', 'MEDCOST', 'SEX', 'CHCKIDNY', 'CVDSTRK3', 'EXERANY2', '_BMI5', 'SMOKE100', '_RFHYPE5', 'DIFFWALK', '_VEGLT1', 'GENHLTH', 'FLUSHOT6', 'USEEQUIP', '_SMOKER3', '_AGEG5YR', 'EDUCA', 'DECIDE', '_CHOLCHK', 'RENTHOM1', 'DIABETE3', 'X_EDUCAG', 'MENTHLTH', '_RFDRHV5', 'TOLDHI2', 'CVDCRHD4', 'X_BMI5CAT', '_TOTINDA', 'ADDEPEV2', 'CHECKUP1', 'EMPLOY1', 'MSCODE', '_MICHD', 'X_TOTINDA', '_FRTLT1', 'BLIND', 'X_AGEG5YR', 'MARITAL', 'HLTHPLN1', 'INCOME2', 'PHYSHLTH', 'X_RACE', 'HLTHCVR1']


## Import and clean datasets

Since there are very few common features shared by 2014 and 2015, I recommend to use the union of those features just for now. We can see how all the data look like after applying all features. Codes below are adapted from [this kaggle notebook](https://www.kaggle.com/alexteboul/diabetes-health-indicators-dataset-notebook). 

In [ ]:
import os
import pandas as pd
import numpy as np
import random
random.seed(1)

In [ ]:
#read in the dataset (2015)
chunks = pd.read_csv(data_path + '2015.csv', chunksize=50000)
brfss_2015_dataset = pd.concat(chunks)
#How many rows and columns
print('2015: ', brfss_2015_dataset.shape)

#read in the dataset (2014)
chunks = pd.read_csv(data_path + '2014.csv', chunksize=50000)
brfss_2014_dataset = pd.concat(chunks)
#How many rows and columns
print('2014: ', brfss_2014_dataset.shape)

#read in the dataset (2013)
chunks = pd.read_csv(data_path + '2013.csv', chunksize=50000)
brfss_2013_dataset = pd.concat(chunks)
#How many rows and columns
print('2013: ', brfss_2013_dataset.shape)

#read in the dataset (2012)
chunks = pd.read_csv(data_path + '2012.csv', chunksize=50000)
brfss_2012_dataset = pd.concat(chunks)
#How many rows and columns
print('2012: ', brfss_2012_dataset.shape)

#read in the dataset (2011)
chunks = pd.read_csv(data_path + '2011.csv', chunksize=50000)
brfss_2011_dataset = pd.concat(chunks)
#How many rows and columns
print('2011: ', brfss_2011_dataset.shape)

2015:  (441456, 330)
2014:  (464664, 279)
2013:  (491773, 336)
2012:  (475687, 359)
2011:  (506467, 454)


In [ ]:
#check that the data loaded in is in the correct format
pd.set_option('display.max_columns', 500)
brfss_2015_dataset.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,PVTRESD1,COLGHOUS,STATERES,CELLFON3,LADULT,NUMADULT,NUMMEN,NUMWOMEN,CTELNUM1,CELLFON2,CADULT,PVTRESD2,CCLGHOUS,CSTATE,LANDLINE,HHADULT,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,HLTHPLN1,PERSDOC2,MEDCOST,CHECKUP1,BPHIGH4,BPMEDS,BLOODCHO,CHOLCHK,TOLDHI2,CVDINFR4,CVDCRHD4,CVDSTRK3,ASTHMA3,ASTHNOW,CHCSCNCR,CHCOCNCR,CHCCOPD1,HAVARTH3,ADDEPEV2,CHCKIDNY,DIABETE3,DIABAGE2,SEX,MARITAL,EDUCA,RENTHOM1,NUMHHOL2,NUMPHON2,CPDEMO1,VETERAN3,EMPLOY1,CHILDREN,INCOME2,INTERNET,WEIGHT2,HEIGHT3,PREGNANT,QLACTLM2,USEEQUIP,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,SMOKE100,SMOKDAY2,STOPSMK2,LASTSMK2,USENOW3,ALCDAY5,AVEDRNK2,DRNK3GE5,MAXDRNKS,FRUITJU1,FRUIT1,FVBEANS,FVGREEN,FVORANG,VEGETAB1,EXERANY2,EXRACT11,EXEROFT1,EXERHMM1,EXRACT21,EXEROFT2,EXERHMM2,STRENGTH,LMTJOIN3,ARTHDIS2,ARTHSOCL,JOINPAIN,SEATBELT,FLUSHOT6,FLSHTMY2,IMFVPLAC,PNEUVAC3,HIVTST6,HIVTSTD3,WHRTST10,PDIABTST,PREDIAB1,INSULIN,BLDSUGAR,FEETCHK2,DOCTDIAB,CHKHEMO3,FEETCHK,EYEEXAM,DIABEYE,DIABEDU,PAINACT2,QLMENTL2,QLSTRES2,QLHLTH2,CAREGIV1,CRGVREL1,CRGVLNG1,CRGVHRS1,CRGVPRB1,CRGVPERS,CRGVHOUS,CRGVMST2,CRGVEXPT,VIDFCLT2,VIREDIF3,VIPRFVS2,VINOCRE2,VIEYEXM2,VIINSUR2,VICTRCT4,VIGLUMA2,VIMACDG2,CIMEMLOS,CDHOUSE,CDASSIST,CDHELP,CDSOCIAL,CDDISCUS,WTCHSALT,LONGWTCH,DRADVISE,ASTHMAGE,ASATTACK,ASERVIST,ASDRVIST,ASRCHKUP,ASACTLIM,ASYMPTOM,ASNOSLEP,ASTHMED3,ASINHALR,HAREHAB1,STREHAB1,CVDASPRN,ASPUNSAF,RLIVPAIN,RDUCHART,RDUCSTRK,ARTTODAY,ARTHWGT,ARTHEXER,ARTHEDU,TETANUS,HPVADVC2,HPVADSHT,SHINGLE2,HADMAM,HOWLONG,HADPAP2,LASTPAP2,HPVTEST,HPLSTTST,HADHYST2,PROFEXAM,LENGEXAM,BLDSTOOL,LSTBLDS3,HADSIGM3,HADSGCO1,LASTSIG3,PCPSAAD2,PCPSADI1,PCPSARE1,PSATEST1,PSATIME,PCPSARS1,PCPSADE1,PCDMDECN,SCNTMNY1,SCNTMEL1,SCNTPAID,SCNTWRK1,SCNTLPAD,SCNTLWK1,SXORIENT,TRNSGNDR,RCSGENDR,RCSRLTN2,CASTHDX2,CASTHNO2,EMTSUPRT,LSATISFY,ADPLEASR,ADDOWN,ADSLEEP,ADENERGY,ADEAT1,ADFAIL,ADTHINK,ADMOVE,MISTMNT,ADANXEV,QSTVER,QSTLANG,EXACTOT1,EXACTOT2,MSCODE,_STSTR,_STRWT,_RAWRAKE,_WT2RAKE,_CHISPNC,_CRACE1,_CPRACE,_CLLCPWT,_DUALUSE,_DUALCOR,_LLCPWT,_RFHLTH,_HCVU651,_RFHYPE5,_CHOLCHK,_RFCHOL,_MICHD,_LTASTH1,_CASTHM1,_ASTHMS1,_DRDXAR1,_PRACE1,_MRACE1,_HISPANC,_RACE,_RACEG21,_RACEGR3,_RACE_G1,_AGEG5YR,_AGE65YR,_AGE80,_AGE_G,HTIN4,HTM4,WTKG3,_BMI5,_BMI5CAT,_RFBMI5,_CHLDCNT,_EDUCAG,_INCOMG,_SMOKER3,_RFSMOK3,DRNKANY5,DROCDY3_,_RFBING5,_DRNKWEK,_RFDRHV5,FTJUDA1_,FRUTDA1_,BEANDAY_,GRENDAY_,ORNGDAY_,VEGEDA1_,_MISFRTN,_MISVEGN,_FRTRESP,_VEGRESP,_FRUTSUM,_VEGESUM,_FRTLT1,_VEGLT1,_FRT16,_VEG23,_FRUITEX,_VEGETEX,_TOTINDA,METVL11_,METVL21_,MAXVO2_,FC60_,ACTIN11_,ACTIN21_,PADUR1_,PADUR2_,PAFREQ1_,PAFREQ2_,_MINAC11,_MINAC21,STRFREQ_,PAMISS1_,PAMIN11_,PAMIN21_,PA1MIN_,PAVIG11_,PAVIG21_,PA1VIGM_,_PACAT1,_PAINDX1,_PA150R2,_PA300R2,_PA30021,_PASTRNG,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,b'01292015',b'01',b'29',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,1.0,NaN,1.0,2.0,NaN,3.0,1.000000e+00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,15.0,18.0,10.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,3.0,NaN,2.0,1.0,4.0,1.0,2.0,NaN,1.0,2.0,8.0,88.0,3.0,2.0,280.0,510.0,NaN,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,NaN,2.0,3.0,888.0,NaN,NaN,NaN,305.0,310.0,320.0,310.0,305.0,101.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,888.0,1.0,1.0,1.0,6.0,1.0,1.0,112014.0,1.0,1.0,1.0,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1.0,b'',b'',3.0,11011.0,28.78156,3.0,86.344681,NaN,NaN,NaN,NaN,1.0,0.614125,341.384853,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,9.0,1.0,63.0,5.0,70.0,178.0,12701.0,4018.0,4.0,2.0,1.0,2.0

In [ ]:
feat_union_1415 = np.sort(np.array(feat_union_1415))
print('selected features shared by 2014 & 2015 datasets, i.e., feat_union_1415: ', len(feat_union_1415))
print(feat_union_1415)
print()

# check the same/different features shared by feat_union_1415 and the 2015 dataset
cols_2015 = brfss_2015_dataset.columns.to_numpy()
feat_shared_2015 = np.intersect1d(feat_union_1415, cols_2015)
print('dataset 2015, common features with feat_union_1415: ', len(feat_shared_2015))
# print(feat_shared_2015)
feat_diff_2015 = np.setdiff1d(feat_union_1415, cols_2015)
print('dataset 2015, different features from feat_union_1415: ', len(feat_diff_2015))
print(feat_diff_2015)
print()

# check the same/different features shared by feat_union_1415 and the 2014 dataset
cols_2014 = brfss_2014_dataset.columns.to_numpy()
feat_shared_2014 = np.intersect1d(feat_union_1415, cols_2014)
print('dataset 2014, common features with feat_union_1415: ', len(feat_shared_2014))
# print(feat_shared_2014)
feat_diff_2014 = np.setdiff1d(feat_union_1415, cols_2014)
print('dataset 2014, different features from feat_union_1415: ', len(feat_diff_2014))
print(feat_diff_2014)
print()

# check the same/different features shared by feat_union_1415 and the 2013 dataset
cols_2013 = brfss_2013_dataset.columns.to_numpy()
feat_shared_2013 = np.intersect1d(feat_union_1415, cols_2013)
print('dataset 2013, common features with feat_union_1415: ', len(feat_shared_2013))
# print(feat_shared_2013)
feat_diff_2013 = np.setdiff1d(feat_union_1415, cols_2013)
print('dataset 2013, different features from feat_union_1415: ', len(feat_diff_2013))
print(feat_diff_2013)
print()

# check the same/different features shared by feat_union_1415 and the 2012 dataset
cols_2012 = brfss_2012_dataset.columns.to_numpy()
feat_shared_2012 = np.intersect1d(feat_union_1415, cols_2012)
print('dataset 2012, common features with feat_union_1415: ', len(feat_shared_2012))
# print(feat_shared_2012)
feat_diff_2012 = np.setdiff1d(feat_union_1415, cols_2012)
print('dataset 2015, different features from feat_union_1415: ', len(feat_diff_2012))
print(feat_diff_2012)
print()

# check the same/different features shared by feat_union_1415 and the 2011 dataset
cols_2011 = brfss_2011_dataset.columns.to_numpy()
feat_shared_2011 = np.intersect1d(feat_union_1415, cols_2011)
print('dataset 2011, common features with feat_union_1415: ', len(feat_shared_2011))
# print(feat_shared_2011)
feat_diff_2011 = np.setdiff1d(feat_union_1415, cols_2011)
print('dataset 2011, different features from feat_union_1415: ', len(feat_diff_2011))
print(feat_diff_2011)
print()

# check the same/different features shared by feat_union_1415 and the 2011-2015 dataset
feat_shared_all = np.intersect1d(np.intersect1d(np.intersect1d(np.intersect1d(np.intersect1d(feat_union_1415, cols_2011), cols_2012), \
                                 cols_2013), cols_2014), cols_2015)
print('dataset 2011-2015, common features with feat_union_1415: ', len(feat_shared_all))
print(feat_shared_all)
feat_diff_all = np.setdiff1d(feat_union_1415, feat_shared_all)
print()
print('dataset 2011-2015, different features from feat_union_1415: ', len(feat_diff_all))
print(feat_diff_all)
print()

if len(feat_shared_all) == len(np.intersect1d(feat_shared_all, feat_union_1415)) and \
len(feat_shared_all) == len(np.intersect1d(feat_shared_all, cols_2011)) and \
len(feat_shared_all) == len(np.intersect1d(feat_shared_all, cols_2012)) and \
len(feat_shared_all) == len(np.intersect1d(feat_shared_all, cols_2013)) and \
len(feat_shared_all) == len(np.intersect1d(feat_shared_all, cols_2014)) and \
len(feat_shared_all) == len(np.intersect1d(feat_shared_all, cols_2015)):
    print('feat_shared_all are in all datasets.')

selected features shared by 2014 & 2015 datasets, i.e., feat_union_1415:  43
['ADDEPEV2' 'BLIND' 'CHCKIDNY' 'CHECKUP1' 'CVDCRHD4' 'CVDSTRK3' 'DECIDE'
 'DIABETE3' 'DIFFWALK' 'EDUCA' 'EMPLOY1' 'EXERANY2' 'FLUSHOT6' 'GENHLTH'
 'HLTHCVR1' 'HLTHPLN1' 'INCOME2' 'MARITAL' 'MEDCOST' 'MENTHLTH' 'MSCODE'
 'PHYSHLTH' 'RENTHOM1' 'SEX' 'SLEPTIM1' 'SMOKE100' 'TOLDHI2' 'USEEQUIP'
 'X_AGEG5YR' 'X_BMI5CAT' 'X_EDUCAG' 'X_RACE' 'X_TOTINDA' '_AGEG5YR'
 '_BMI5' '_CHOLCHK' '_FRTLT1' '_MICHD' '_RFDRHV5' '_RFHYPE5' '_SMOKER3'
 '_TOTINDA' '_VEGLT1']

dataset 2015, common features with feat_union_1415:  36
dataset 2015, different features from feat_union_1415:  7
['HLTHCVR1' 'SLEPTIM1' 'X_AGEG5YR' 'X_BMI5CAT' 'X_EDUCAG' 'X_RACE'
 'X_TOTINDA']

dataset 2014, common features with feat_union_1415:  31
dataset 2014, different features from feat_union_1415:  12
['TOLDHI2' 'X_AGEG5YR' 'X_BMI5CAT' 'X_EDUCAG' 'X_RACE' 'X_TOTINDA'
 '_CHOLCHK' '_FRTLT1' '_MICHD' '_RFDRHV5' '_RFHYPE5' '_VEGLT1']

dataset 2013, common feat

### Clean dataset 2015

Below are the cleaning process of the 2015 dataset, using features in feat_shared_2015. The values of each feature can be found in [this notebook](https://www.kaggle.com/alexteboul/diabetes-health-indicators-dataset-notebook) and [this paper](https://www.cdc.gov/pcd/issues/2019/19_0109.htm). 

In [ ]:
# select specific columns
brfss_2015_selected = brfss_2015_dataset[feat_shared_all]

In [ ]:
print('2015 dataset shape, initial: ', brfss_2015_selected.shape)

2015 dataset shape, initial:  (441456, 24)


In [ ]:
#Drop Missing Values
brfss_2015_selected = brfss_2015_selected.dropna()
print('2015 dataset shape, after dropping nan: ', brfss_2015_selected.shape)

2015 dataset shape, after dropping nan:  (217353, 24)


In [ ]:
# 1 DIABETE3: (Ever told) you have diabetes. 
# 0 is for no diabetes or only during pregnancy, 1 is for pre-diabetes or borderline diabetes and yes diabetes
# Remove all 7 (dont knows)
# Remove all 9 (refused)
brfss_2015_selected['DIABETE3'] = brfss_2015_selected['DIABETE3'].replace({2:0, 3:0, 4:1})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.DIABETE3 != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.DIABETE3 != 9]
print('2015 dataset, DIABETE3 values: ', brfss_2015_selected.DIABETE3.unique())
print('2015 dataset shape, after cleaning DISBETE3: ', brfss_2015_selected.shape)

2015 dataset, DIABETE3 values:  [0. 1.]
2015 dataset shape, after cleaning DISBETE3:  (217076, 24)


In [ ]:
# 2 _BMI5: Body Mass Index (BMI) 
# (no changes, just note that these are BMI * 100. So for example a BMI of 4018 is really 40.18)
brfss_2015_selected['_BMI5'] = brfss_2015_selected['_BMI5'].div(100).round(0)
print('2015 dataset, _BMI5 values: ', brfss_2015_selected._BMI5.unique())
print('2015 dataset shape, after cleaning _BMI5: ', brfss_2015_selected.shape)

2015 dataset, _BMI5 values:  [ 40.  25.  28.  24.  27.  34.  15.  30.  26.  31.  23.  51.  14.  33.
  21.  22.  35.  38.  20.  19.  32.  37.  36.  29.  39.  18.  45.  47.
  16.  43.  55.  49.  42.  17.  41.  48.  44.  50.  59.  46.  52.  54.
  53.  57.  60.  74.  56.  62.  13.  58.  66.  65.  68.  67.  85.  72.
  61.  63.  64.  71.  70.  79.  81.  69.  75.  12.  78.  82.  76.  73.
  77.  87.  95.  89.  80.  90.  86.  84. 100.  97.  88.]
2015 dataset shape, after cleaning _BMI5:  (217076, 24)


In [ ]:
# 3 SMOKE100: Have you smoked at least 100 cigarettes in your entire life? [Note: 5 packs = 100 cigarettes]
# Change 2 to 0 because it is No
# Remove all 7 (dont knows)
# Remove all 9 (refused)
brfss_2015_selected['SMOKE100'] = brfss_2015_selected['SMOKE100'].replace({2:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.SMOKE100 != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.SMOKE100 != 9]
print('2015 dataset, SMOKE100 values: ', brfss_2015_selected.SMOKE100.unique())
print('2015 dataset shape, after cleaning SMOKE100: ', brfss_2015_selected.shape)

2015 dataset, SMOKE100 values:  [1. 0.]
2015 dataset shape, after cleaning SMOKE100:  (215826, 24)


In [ ]:
# 4 CVDSTRK3: (Ever told) you had a stroke.
# Change 2 to 0 because it is No
# Remove all 7 (dont knows)
# Remove all 9 (refused)
brfss_2015_selected['CVDSTRK3'] = brfss_2015_selected['CVDSTRK3'].replace({2:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.CVDSTRK3 != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.CVDSTRK3 != 9]
print('2015 dataset, CVDSTRK3 values: ', brfss_2015_selected.CVDSTRK3.unique())
print('2015 dataset shape, after cleaning CVDSTRK3: ', brfss_2015_selected.shape)

2015 dataset, CVDSTRK3 values:  [0. 1.]
2015 dataset shape, after cleaning CVDSTRK3:  (215206, 24)


In [ ]:
# 5 _TOTINDA: Adults who reported doing physical activity or exercise during the past 30 days other than their regular job
# 1 for physical activity
# change 2 to 0 for no physical activity
# Remove all 9 (don't know/refused)
brfss_2015_selected['_TOTINDA'] = brfss_2015_selected['_TOTINDA'].replace({2:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected._TOTINDA != 9]
print('2015 dataset, _TOTINDA values: ', brfss_2015_selected._TOTINDA.unique())
print('2015 dataset shape, after cleaning _TOTIDNA: ', brfss_2015_selected.shape)

2015 dataset, _TOTINDA values:  [0. 1.]
2015 dataset shape, after cleaning _TOTIDNA:  (214653, 24)


In [ ]:
# 6 HLTHPLN1: Do you have any kind of health care coverage, 
# including health insurance, prepaid plans such as HMOs, or government plans such as Medicare, or Indian Health Service?
# 1 is yes, change 2 to 0 because it is No health care access
# remove 7 and 9 for don't know or refused
brfss_2015_selected['HLTHPLN1'] = brfss_2015_selected['HLTHPLN1'].replace({2:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.HLTHPLN1 != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.HLTHPLN1 != 9]
print('2015 dataset, HLTHPLN1 values: ', brfss_2015_selected.HLTHPLN1.unique())
print('2015 dataset shape, after cleaning HLTHPLN1: ', brfss_2015_selected.shape)

2015 dataset, HLTHPLN1 values:  [1. 0.]
2015 dataset shape, after cleaning HLTHPLN1:  (214111, 24)


In [ ]:
# 7 MEDCOST: Was there a time in the past 12 months when you needed to see a doctor but could not because of cost?
# Change 2 to 0 for no, 1 is already yes
# remove 7 for don/t know and 9 for refused
brfss_2015_selected['MEDCOST'] = brfss_2015_selected['MEDCOST'].replace({2:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.MEDCOST != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.MEDCOST != 9]
print('2015 dataset, MEDCOST values: ', brfss_2015_selected.MEDCOST.unique())
print('2015 dataset shape, after cleaning MEDCOST: ', brfss_2015_selected.shape)

2015 dataset, MEDCOST values:  [0. 1.]
2015 dataset shape, after cleaning MEDCOST:  (213643, 24)


In [ ]:
# 8 GENHLTH: Would you say that in general your health is:
# This is an ordinal variable that I want to keep (1 is Excellent -> 5 is Poor)
# Remove 7 and 9 for don't know and refused
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.GENHLTH != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.GENHLTH != 9]
print('2015 dataset, GENHLTH values: ', brfss_2015_selected.GENHLTH.unique())
print('2015 dataset shape, after cleaning GENHLTH: ', brfss_2015_selected.shape)

2015 dataset, GENHLTH values:  [5. 3. 2. 4. 1.]
2015 dataset shape, after cleaning GENHLTH:  (213061, 24)


In [ ]:
# 9 MENTHLTH: Now thinking about your mental health, which includes stress, 
# depression, and problems with emotions, for how many days during the past 30 days was your mental health not good?
# already in days so keep that, scale will be 0-30
# change 88 to 0 because it means none (no bad mental health days)
# remove 77 and 99 for don't know not sure and refused
brfss_2015_selected['MENTHLTH'] = brfss_2015_selected['MENTHLTH'].replace({88:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.MENTHLTH != 77]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.MENTHLTH != 99]
print('2015 dataset, MENTHLTH values: ', brfss_2015_selected.MENTHLTH.unique())
print('2015 dataset shape, after cleaning MENTHLTH: ', brfss_2015_selected.shape)

2015 dataset, MENTHLTH values:  [18.  0. 30.  3.  2.  5. 15. 10.  1.  6. 20. 25. 29.  4.  7.  8. 21. 14.
 26.  9. 16. 28. 11. 12. 24. 17. 13. 19. 22. 27. 23.]
2015 dataset shape, after cleaning MENTHLTH:  (209655, 24)


In [ ]:
# 10 PHYSHLTH: Now thinking about your physical health,
#  which includes physical illness and injury, for how many days during the past 30 days was your physical health not good?
# already in days so keep that, scale will be 0-30
# change 88 to 0 because it means none (no bad physical health days)
# remove 77 and 99 for don't know not sure and refused
brfss_2015_selected['PHYSHLTH'] = brfss_2015_selected['PHYSHLTH'].replace({88:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.PHYSHLTH != 77]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.PHYSHLTH != 99]
print('2015 dataset, PHYSHLTH values: ', brfss_2015_selected.PHYSHLTH.unique())
print('2015 dataset shape, after cleaning PHYSHLTH: ', brfss_2015_selected.shape)

2015 dataset, PHYSHLTH values:  [15.  0. 30. 20.  8.  2. 14.  6. 28.  1.  7. 10.  3.  5. 17.  4. 19. 21.
 25. 12. 27. 22. 29. 24.  9. 16. 13. 18. 23. 26. 11.]
2015 dataset shape, after cleaning PHYSHLTH:  (205694, 24)


In [ ]:
# 11 SEX: Indicate sex of respondent.
# in other words - is respondent male (somewhat arbitrarily chose this change because men are at higher risk for heart disease)
# change 2 to 0 (female as 0). Male is 1
brfss_2015_selected['SEX'] = brfss_2015_selected['SEX'].replace({2:0})
print('2015 dataset, SEX values: ', brfss_2015_selected.SEX.unique())
print('2015 dataset shape, after cleaning SEX: ', brfss_2015_selected.shape)

2015 dataset, SEX values:  [0. 1.]
2015 dataset shape, after cleaning SEX:  (205694, 24)


In [ ]:
# 12 _AGEG5YR: Fourteen-level age category
# already ordinal. 1 is 18-24 all the way up to 13 wis 80 and older. 5 year increments.
# remove 14 because it is don't know or missing
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected._AGEG5YR != 14]
print('2015 dataset, _AGEG5YR values: ', brfss_2015_selected._AGEG5YR.unique())
print('2015 dataset shape, after cleaning _AGEG5YR: ', brfss_2015_selected.shape)

2015 dataset, _AGEG5YR values:  [ 9.  7. 11. 13. 10.  8.  4.  6.  2.  3. 12.  5.  1.]
2015 dataset shape, after cleaning _AGEG5YR:  (204519, 24)


In [ ]:
# 13 EDUCA: What is the highest grade or year of school you completed?
# This is already an ordinal variable with 1 being never attended school or kindergarten only up to 6 being college 4 years or more
# Scale here is 1-6
# Remove 9 for refused:
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.EDUCA != 9]
print('2015 dataset, EDUCA values: ', brfss_2015_selected.EDUCA.unique())
print('2015 dataset shape, after cleaning EDUCA: ', brfss_2015_selected.shape)

2015 dataset, EDUCA values:  [4. 6. 5. 3. 2. 1.]
2015 dataset shape, after cleaning EDUCA:  (204187, 24)


In [ ]:
# 14 INCOME2: Is your annual household income from all sources: (If respondent refuses at any income level, code "Refused.")
# Variable is already ordinal with 1 being less than $10,000 all the way up to 8 being $75,000 or more
# Remove 77 and 99 for don't know and refused
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.INCOME2 != 77]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.INCOME2 != 99]
print('2015 dataset, INCOME2 values: ', brfss_2015_selected.INCOME2.unique())
print('2015 dataset shape, after cleaning INCOME2: ', brfss_2015_selected.shape)

2015 dataset, INCOME2 values:  [3. 1. 8. 6. 4. 7. 2. 5.]
2015 dataset shape, after cleaning INCOME2:  (173746, 24)


Variables below are from the 2014 paper.

In [ ]:
# 15 ADDEPEV2: (Ever told) you that you have a depressive disorder, including depression, major depression, dysthymia, or minor depression?
# change 2 (No) to 0
# remove 7 (Don't know/Not sure) and 9 (Refused)
brfss_2015_selected['ADDEPEV2'] = brfss_2015_selected['ADDEPEV2'].replace({2:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.ADDEPEV2 != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.ADDEPEV2 != 9]
print('2015 dataset, ADDEPEV2 values: ', brfss_2015_selected.ADDEPEV2.unique())
print('2015 dataset shape, after cleaning ADDEPEV2: ', brfss_2015_selected.shape)

2015 dataset, ADDEPEV2 values:  [1. 0.]
2015 dataset shape, after cleaning ADDEPEV2:  (173302, 24)


In [ ]:
# 16 CHCKIDNY: (Ever told) you have kidney disease? Do NOT include kidney stones, 
# bladder infection or incontinence.(Incontinence is not being able to control urine flow.)
# change 2 (No) to 0
# remove 7 (Don't know/Not sure) and 9 (Refused)
brfss_2015_selected['CHCKIDNY'] = brfss_2015_selected['CHCKIDNY'].replace({2:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.CHCKIDNY != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.CHCKIDNY != 9]
print('2015 dataset, CHCKIDNY values: ', brfss_2015_selected.CHCKIDNY.unique())
print('2015 dataset shape, after cleaning CHCKIDNY: ', brfss_2015_selected.shape)

2015 dataset, CHCKIDNY values:  [0. 1.]
2015 dataset shape, after cleaning CHCKIDNY:  (172832, 24)


In [ ]:
# 17 CHECKUP1: About how long has it been since you last visited a doctor for a routine checkup? 
# [A routine checkup is a general physical exam, not an exam for a specific injury, illness, or condition.]
# change 8 (Never) to 0
# remove 7 (Don't know/Not sure) and 9 (Refused)
brfss_2015_selected['CHECKUP1'] = brfss_2015_selected['CHECKUP1'].replace({8:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.CHECKUP1 != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.CHECKUP1 != 9]
print('2015 dataset, CHECKUP1 values: ', brfss_2015_selected.CHECKUP1.unique())
print('2015 dataset shape, after cleaning CHECKUP1: ', brfss_2015_selected.shape)

2015 dataset, CHECKUP1 values:  [1. 4. 2. 3. 0.]
2015 dataset shape, after cleaning CHECKUP1:  (171439, 24)


In [ ]:
# 18 CVDCRHD4: (Ever told) you had angina or coronary heart disease?
# [A routine checkup is a general physical exam, not an exam for a specific injury, illness, or condition.]
# change 8 (Never) to 0
# remove 7 (Don't know/Not sure) and 9 (Refused)
brfss_2015_selected['CVDCRHD4'] = brfss_2015_selected['CVDCRHD4'].replace({2:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.CVDCRHD4 != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.CVDCRHD4 != 9]
print('2015 dataset, CVDCRHD4 values: ', brfss_2015_selected.CVDCRHD4.unique())
print('2015 dataset shape, after cleaning CVDCRHD4: ', brfss_2015_selected.shape)

2015 dataset, CVDCRHD4 values:  [0. 1.]
2015 dataset shape, after cleaning CVDCRHD4:  (170160, 24)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# 19 EXERANY2: During the past month, other than your regular job, 
# did you participate in any physical activities or exercises such as running, calisthenics, golf, gardening, or walking for exercise?
# change 8 (Never) to 0
# remove 7 (Don't know/Not sure) and 9 (Refused)
brfss_2015_selected['EXERANY2'] = brfss_2015_selected['EXERANY2'].replace({2:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.EXERANY2 != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.EXERANY2 != 9]
print('2015 dataset, EXERANY2 values: ', brfss_2015_selected.EXERANY2.unique())
print('2015 dataset shape, after cleaning EXERANY2: ', brfss_2015_selected.shape)

2015 dataset, EXERANY2 values:  [0. 1.]
2015 dataset shape, after cleaning EXERANY2:  (170160, 24)


In [ ]:
# 20 MARITAL: Are you: (marital status)
# keep the ordinal order
# remove 9 (Refused)
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.MARITAL != 9]
print('2015 dataset, MARITAL values: ', brfss_2015_selected.MARITAL.unique())
print('2015 dataset shape, after cleaning MARITAL: ', brfss_2015_selected.shape)

2015 dataset, MARITAL values:  [1. 2. 3. 6. 5. 4.]
2015 dataset shape, after cleaning MARITAL:  (169826, 24)


In [ ]:
# 21 MSCODE: Metropolitan Status Code
# change 5 (Not in an MSA) to 0.
brfss_2015_selected['MSCODE'] = brfss_2015_selected['MSCODE'].replace({5:0})
print('2015 dataset, MSCODE values: ', brfss_2015_selected.MSCODE.unique())
print('2015 dataset shape, after cleaning MSCODE: ', brfss_2015_selected.shape)

2015 dataset, MSCODE values:  [3. 0. 1. 2.]
2015 dataset shape, after cleaning MSCODE:  (169826, 24)


In [ ]:
# 22 RENTHOM1: Do you own or rent your home? (Home is defined as the place where you live most of the time/the majority of the year.)
# remove 7 (Don't know/Not sure) and 9 (Refused)
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.RENTHOM1 != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.RENTHOM1 != 9]
print('2015 dataset, RENTHOM1 values: ', brfss_2015_selected.RENTHOM1.unique())
print('2015 dataset shape, after cleaning RENTHOM1: ', brfss_2015_selected.shape)

2015 dataset, RENTHOM1 values:  [1. 2. 3.]
2015 dataset shape, after cleaning RENTHOM1:  (169568, 24)


In [ ]:
# 23 USEEQUIP: Do you now have any health problem that requires you to use special equipment,\
# such as a cane, a wheelchair, a special bed, or a special telephone? (Include occasional use or use in certain circumstances.)
# change 2 (No) to 0
# remove 7 (Don't know/Not sure) and 9 (Refused)
brfss_2015_selected['USEEQUIP'] = brfss_2015_selected['USEEQUIP'].replace({2:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.USEEQUIP != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected.USEEQUIP != 9]
print('2015 dataset, USEEQUIP: ', brfss_2015_selected.USEEQUIP.unique())
print('2015 dataset shape, after cleaning USEEQUIP: ', brfss_2015_selected.shape)

2015 dataset, USEEQUIP:  [1. 0.]
2015 dataset shape, after cleaning USEEQUIP:  (169456, 24)


In [ ]:
# 24 _SMOKER3: Four-level smoker status: Everyday smoker, Someday smoker, Former smoker, Non-smoker
# change 4 (Never smoked) to 0
# remove 9 (Don’t know/Refused/Missing)
brfss_2015_selected['_SMOKER3'] = brfss_2015_selected['_SMOKER3'].replace({4:0})
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected._SMOKER3 != 7]
brfss_2015_selected = brfss_2015_selected[brfss_2015_selected._SMOKER3 != 9]
print('2015 dataset, _SMOKER3: ', brfss_2015_selected._SMOKER3.unique())
print('2015 dataset shape, after cleaning _SMOKER3: ', brfss_2015_selected.shape)

2015 dataset, _SMOKER3:  [3. 1. 0. 2.]
2015 dataset shape, after cleaning _SMOKER3:  (169386, 24)


In [ ]:
brfss_2015_selected.to_csv(data_path + 'pre_processed/2015_selected.csv')

### Repeat the process above for datasets 2011-2014

In [ ]:
def clean_dataset(dataframe, columns, year):
    # select specific columns
    df = dataframe[columns]
    print('----------------------------------------------------------------')
    print(f'--------------- Start cleaning {year} dataset --------------------')
    print('----------------------------------------------------------------')
    print()
    print(f'{year} dataset shape, initial: ', df.shape)
    print()

    #Drop Missing Values
    df = df.dropna()
    print(f'{year} dataset shape, after dropping nan: ', df.shape)
    print()

    # 1 DIABETE3: (Ever told) you have diabetes. 
    # 0 is for no diabetes or only during pregnancy, 1 is for pre-diabetes or borderline diabetes and yes diabetes
    # Remove all 7 (dont knows)
    # Remove all 9 (refused)
    df['DIABETE3'] = df['DIABETE3'].replace({2:0, 3:0, 4:1})
    df = df[df.DIABETE3 != 7]
    df = df[df.DIABETE3 != 9]
    print(f'{year} dataset, DIABETE3 values: ', df.DIABETE3.unique())
    print(f'{year} dataset shape, after cleaning DISBETE3: ', df.shape)
    print()

    # 2 _BMI5: Body Mass Index (BMI) 
    # (no changes, just note that these are BMI * 100. So for example a BMI of 4018 is really 40.18)
    df['_BMI5'] = df['_BMI5'].div(100).round(0)
    print(f'{year} dataset, _BMI5 values: ', df._BMI5.unique())
    print(f'{year} dataset shape, after cleaning _BMI5: ', df.shape)
    print()

    # 3 SMOKE100: Have you smoked at least 100 cigarettes in your entire life? [Note: 5 packs = 100 cigarettes]
    # Change 2 to 0 because it is No
    # Remove all 7 (dont knows)
    # Remove all 9 (refused)
    df['SMOKE100'] = df['SMOKE100'].replace({2:0})
    df = df[df.SMOKE100 != 7]
    df = df[df.SMOKE100 != 9]
    print(f'{year} dataset, SMOKE100 values: ', df.SMOKE100.unique())
    print(f'{year} dataset shape, after cleaning SMOKE100: ', df.shape)
    print()

    # 4 CVDSTRK3: (Ever told) you had a stroke.
    # Change 2 to 0 because it is No
    # Remove all 7 (dont knows)
    # Remove all 9 (refused)
    df['CVDSTRK3'] = df['CVDSTRK3'].replace({2:0})
    df = df[df.CVDSTRK3 != 7]
    df = df[df.CVDSTRK3 != 9]
    print(f'{year} dataset, CVDSTRK3 values: ', df.CVDSTRK3.unique())
    print(f'{year} dataset shape, after cleaning CVDSTRK3: ', df.shape)
    print()

    # 5 _TOTINDA: Adults who reported doing physical activity or exercise during the past 30 days other than their regular job
    # 1 for physical activity
    # change 2 to 0 for no physical activity
    # Remove all 9 (don't know/refused)
    df['_TOTINDA'] = df['_TOTINDA'].replace({2:0})
    df = df[df._TOTINDA != 9]
    print(f'{year} dataset, _TOTINDA values: ', df._TOTINDA.unique())
    print(f'{year} dataset shape, after cleaning _TOTIDNA: ', df.shape)
    print()

    # 6 HLTHPLN1: Do you have any kind of health care coverage, 
    # including health insurance, prepaid plans such as HMOs, or government plans such as Medicare, or Indian Health Service?
    # 1 is yes, change 2 to 0 because it is No health care access
    # remove 7 and 9 for don't know or refused
    df['HLTHPLN1'] = df['HLTHPLN1'].replace({2:0})
    df = df[df.HLTHPLN1 != 7]
    df = df[df.HLTHPLN1 != 9]
    print(f'{year} dataset, HLTHPLN1 values: ', df.HLTHPLN1.unique())
    print(f'{year} dataset shape, after cleaning HLTHPLN1: ', df.shape)
    print()

    # 7 MEDCOST: Was there a time in the past 12 months when you needed to see a doctor but could not because of cost?
    # Change 2 to 0 for no, 1 is already yes
    # remove 7 for don/t know and 9 for refused
    df['MEDCOST'] = df['MEDCOST'].replace({2:0})
    df = df[df.MEDCOST != 7]
    df = df[df.MEDCOST != 9]
    print(f'{year} dataset, MEDCOST values: ', df.MEDCOST.unique())
    print(f'{year} dataset shape, after cleaning MEDCOST: ', df.shape)
    print()

    # 8 GENHLTH: Would you say that in general your health is:
    # This is an ordinal variable that I want to keep (1 is Excellent -> 5 is Poor)
    # Remove 7 and 9 for don't know and refused
    df = df[df.GENHLTH != 7]
    df = df[df.GENHLTH != 9]
    print(f'{year} dataset, GENHLTH values: ', df.GENHLTH.unique())
    print(f'{year} dataset shape, after cleaning GENHLTH: ', df.shape)
    print()

    # 9 MENTHLTH: Now thinking about your mental health, which includes stress, 
    # depression, and problems with emotions, for how many days during the past 30 days was your mental health not good?
    # already in days so keep that, scale will be 0-30
    # change 88 to 0 because it means none (no bad mental health days)
    # remove 77 and 99 for don't know not sure and refused
    df['MENTHLTH'] = df['MENTHLTH'].replace({88:0})
    df = df[df.MENTHLTH != 77]
    df = df[df.MENTHLTH != 99]
    print(f'{year} dataset, MENTHLTH values: ', df.MENTHLTH.unique())
    print(f'{year} dataset shape, after cleaning MENTHLTH: ', df.shape)
    print()

    # 10 PHYSHLTH: Now thinking about your physical health,
    #  which includes physical illness and injury, for how many days during the past 30 days was your physical health not good?
    # already in days so keep that, scale will be 0-30
    # change 88 to 0 because it means none (no bad mental health days)
    # remove 77 and 99 for don't know not sure and refused
    df['PHYSHLTH'] = df['PHYSHLTH'].replace({88:0})
    df = df[df.PHYSHLTH != 77]
    df = df[df.PHYSHLTH != 99]
    print(f'{year} dataset, PHYSHLTH values: ', df.PHYSHLTH.unique())
    print(f'{year} dataset shape, after cleaning PHYSHLTH: ', df.shape)
    print()

    # 11 SEX: Indicate sex of respondent.
    # in other words - is respondent male (somewhat arbitrarily chose this change because men are at higher risk for heart disease)
    # change 2 to 0 (female as 0). Male is 1
    df['SEX'] = df['SEX'].replace({2:0})
    print(f'{year} dataset, SEX values: ', df.SEX.unique())
    print(f'{year} dataset shape, after cleaning SEX: ', df.shape)
    print()

    # 12 _AGEG5YR: Fourteen-level age category
    # already ordinal. 1 is 18-24 all the way up to 13 wis 80 and older. 5 year increments.
    # remove 14 because it is don't know or missing
    df = df[df._AGEG5YR != 14]
    print(f'{year} dataset, _AGEG5YR values: ', df._AGEG5YR.unique())
    print(f'{year} dataset shape, after cleaning _AGEG5YR: ', df.shape)
    print()

    # 13 EDUCA: What is the highest grade or year of school you completed?
    # This is already an ordinal variable with 1 being never attended school or kindergarten only up to 6 being college 4 years or more
    # Scale here is 1-6
    # Remove 9 for refused:
    df = df[df.EDUCA != 9]
    print(f'{year} dataset, EDUCA values: ', df.EDUCA.unique())
    print(f'{year} dataset shape, after cleaning EDUCA: ', df.shape)
    print()

    # 14 INCOME2: Is your annual household income from all sources: (If respondent refuses at any income level, code "Refused.")
    # Variable is already ordinal with 1 being less than $10,000 all the way up to 8 being $75,000 or more
    # Remove 77 and 99 for don't know and refused
    df = df[df.INCOME2 != 77]
    df = df[df.INCOME2 != 99]
    print(f'{year} dataset, INCOME2 values: ', df.INCOME2.unique())
    print(f'{year} dataset shape, after cleaning INCOME2: ', df.shape)
    print()

    # 15 ADDEPEV2: (Ever told) you that you have a depressive disorder, including depression, major depression, dysthymia, or minor depression?
    # change 2 (No) to 0
    # remove 7 (Don't know/Not sure) and 9 (Refused)
    df['ADDEPEV2'] = df['ADDEPEV2'].replace({2:0})
    df = df[df.ADDEPEV2 != 7]
    df = df[df.ADDEPEV2 != 9]
    print(f'{year} dataset, ADDEPEV2 values: ', df.ADDEPEV2.unique())
    print(f'{year} dataset shape, after cleaning ADDEPEV2: ', df.shape)
    print()

    # 16 CHCKIDNY: (Ever told) you have kidney disease? Do NOT include kidney stones, 
    # bladder infection or incontinence.(Incontinence is not being able to control urine flow.)
    # change 2 (No) to 0
    # remove 7 (Don't know/Not sure) and 9 (Refused)
    df['CHCKIDNY'] = df['CHCKIDNY'].replace({2:0})
    df = df[df.CHCKIDNY != 7]
    df = df[df.CHCKIDNY != 9]
    print(f'{year} dataset, CHCKIDNY values: ', df.CHCKIDNY.unique())
    print(f'{year} dataset shape, after cleaning CHCKIDNY: ', df.shape)
    print()

    # 17 CHECKUP1: About how long has it been since you last visited a doctor for a routine checkup? 
    # [A routine checkup is a general physical exam, not an exam for a specific injury, illness, or condition.]
    # change 8 (Never) to 0
    # remove 7 (Don't know/Not sure) and 9 (Refused)
    df['CHECKUP1'] = df['CHECKUP1'].replace({8:0})
    df = df[df.CHECKUP1 != 7]
    df = df[df.CHECKUP1 != 9]
    print(f'{year} dataset, CHECKUP1 values: ', df.CHECKUP1.unique())
    print(f'{year} dataset shape, after cleaning CHECKUP1: ', df.shape)
    print()

    # 18 CVDCRHD4: (Ever told) you had angina or coronary heart disease?
    # [A routine checkup is a general physical exam, not an exam for a specific injury, illness, or condition.]
    # change 8 (Never) to 0
    # remove 7 (Don't know/Not sure) and 9 (Refused)
    df['CVDCRHD4'] = df['CVDCRHD4'].replace({2:0})
    df = df[df.CVDCRHD4 != 7]
    df = df[df.CVDCRHD4 != 9]
    print(f'{year} dataset, CVDCRHD4 values: ', df.CVDCRHD4.unique())
    print(f'{year} dataset shape, after cleaning CVDCRHD4: ', df.shape)
    print()

    # 19 EXERANY2: During the past month, other than your regular job, 
    # did you participate in any physical activities or exercises such as running, calisthenics, golf, gardening, or walking for exercise?
    # change 8 (Never) to 0
    # remove 7 (Don't know/Not sure) and 9 (Refused)
    df['EXERANY2'] = df['EXERANY2'].replace({2:0})
    df = df[df.EXERANY2 != 7]
    df = df[df.EXERANY2 != 9]
    print(f'{year} dataset, EXERANY2 values: ', df.EXERANY2.unique())
    print(f'{year} dataset shape, after cleaning EXERANY2: ', df.shape)
    print()

    # 20 MARITAL: Are you: (marital status)
    # keep the ordinal order
    # remove 9 (Refused)
    df = df[df.MARITAL != 9]
    print(f'{year} dataset, MARITAL values: ', df.MARITAL.unique())
    print(f'{year} dataset shape, after cleaning MARITAL: ', df.shape)
    print()

    # 21 MSCODE: Metropolitan Status Code
    # change 5 (Not in an MSA) to 0.
    df['MSCODE'] = df['MSCODE'].replace({5:0})
    print(f'{year} dataset, MSCODE values: ', df.MSCODE.unique())
    print(f'{year} dataset shape, after cleaning MSCODE: ', df.shape)
    print()

    # 22 RENTHOM1: Do you own or rent your home? (Home is defined as the place where you live most of the time/the majority of the year.)
    # remove 7 (Don't know/Not sure) and 9 (Refused)
    df = df[df.RENTHOM1 != 7]
    df = df[df.RENTHOM1 != 9]
    print(f'{year} dataset, RENTHOM1 values: ', df.RENTHOM1.unique())
    print(f'{year} dataset shape, after cleaning RENTHOM1: ', df.shape)
    print()

    # 23 USEEQUIP: Do you now have any health problem that requires you to use special equipment,\
    # such as a cane, a wheelchair, a special bed, or a special telephone? (Include occasional use or use in certain circumstances.)
    # change 2 (No) to 0
    # remove 7 (Don't know/Not sure) and 9 (Refused)
    df['USEEQUIP'] = df['USEEQUIP'].replace({2:0})
    df = df[df.USEEQUIP != 7]
    df = df[df.USEEQUIP != 9]
    print(f'{year} dataset, USEEQUIP: ', df.USEEQUIP.unique())
    print(f'{year} dataset shape, after cleaning USEEQUIP: ', df.shape)
    print()

    # 24 _SMOKER3: Four-level smoker status: Everyday smoker, Someday smoker, Former smoker, Non-smoker
    # change 4 (Never smoked) to 0
    # remove 9 (Don’t know/Refused/Missing)
    df['_SMOKER3'] = df['_SMOKER3'].replace({4:0})
    df = df[df._SMOKER3 != 7]
    df = df[df._SMOKER3 != 9]
    print(f'{year} dataset, _SMOKER3: ', df._SMOKER3.unique())
    print(f'{year} dataset shape, after cleaning _SMOKER3: ', df.shape)
    print()

    return df

In [ ]:
for i in range(4):
    dataframe = [brfss_2011_dataset, brfss_2012_dataset, brfss_2013_dataset, brfss_2014_dataset][i]
    year = [2011, 2012, 2013, 2014][i]
    cleaned_df = clean_dataset(dataframe, feat_shared_all, year)
    cleaned_df.to_csv(data_path + f'pre_processed/{year}_selected.csv')

----------------------------------------------------------------
--------------- Start cleaning 2011 dataset --------------------
----------------------------------------------------------------

2011 dataset shape, initial:  (506467, 24)

2011 dataset shape, after dropping nan:  (388579, 24)

2011 dataset, DIABETE3 values:  [0. 1.]
2011 dataset shape, after cleaning DISBETE3:  (388070, 24)

2011 dataset, _BMI5 values:  [19. 17. 43. 30. 31. 21. 22. 27. 26. 25. 34. 39. 24. 38. 23. 32. 29. 48.
 28. 52. 20. 18. 33. 41. 42. 35. 36. 44. 40. 50. 57. 45. 37. 16. 51. 49.
 47. 77. 15. 46. 56. 53. 83. 65. 14. 60. 59. 55. 70. 78. 61. 98. 58. 54.
 64. 73. 79. 62. 13. 82. 63. 66. 76. 74. 68. 75. 67. 12. 71. 69. 84. 92.
 80. 89. 93. 72. 94. 95. 86. 81.]
2011 dataset shape, after cleaning _BMI5:  (388070, 24)

2011 dataset, SMOKE100 values:  [1. 0.]
2011 dataset shape, after cleaning SMOKE100:  (386407, 24)

2011 dataset, CVDSTRK3 values:  [0. 1.]
2011 dataset shape, after cleaning CVDSTRK3:  (385430